# レポート8前半
1810083 井上悠香

In [ ]:
#70
import sys
sys.path.append(' . . ')
import gensim
import re
import torch
import numpy as np

w2v_path = './GoogleNews-vectors-negative300.bin' #todo GoogleNews-vectors-negative300.binのパスを指定
data_path = '/Users/inoueyuuka/Desktop/kobo2020/data70/'#todo ファイル出力ディレクトリを指定

w2v_model = gensim.models.KeyedVectors.load_word2vec_format(w2v_path, binary=True)#todo gensimでword2vecのモデルを読み込む


In [31]:
category2num = {"b": 0, "t": 1, "e": 2, "m": 3}

# タイトルを受け取り，単語ベクトルの平均を返す関数
def get_feature(title):
    word_list = title.split(' ')#todo タイトルをスペースで分割
    vec_list = []
    for word in word_list:
        try:
            vec = w2v_model[word]#todo wordを意味するベクトルを取得　これで良いはず？
        except KeyError:
            vec = np.zeros(300)#todo すべての要素が0のベクトルを代入 np.zeros()でいいんか？？
        vec_list.append(vec)    
    vec_np = np.array(vec_list) # numpyのarrayに変換
    feature = np.mean(vec_np)# 平均ベクトルを計算
    return feature

def get_data(fname):
    label_list = []
    feature_list = []
    with open(fname) as f:
        for line in f:
            if not line:
                continue
            data = line.split('\t')
            title = data[1]#todo dataからタイトルを取り出す
            feature = get_feature(title) 
            feature_list.append(feature)
            label = category2num[data[0]]
            label_list.append(label)
            
    feature_list = np.array(feature_list)#追加
    features =  torch.from_numpy(feature_list)#todo feature_listをtensorに変換
    label_list = np.array(label_list)#追加
    labels =  torch.from_numpy(label_list)#todo label_listをtensorに変換
    return features, labels


train_x, train_y = get_data(data_path + "train.txt")
valid_x, valid_y = get_data(data_path + "valid.txt")
test_x, test_y = get_data(data_path + "test.txt")

# 保存
torch.save(train_x, data_path + "train_x.pt")
torch.save(train_y, data_path + "train_y.pt")
torch.save(valid_x, data_path + "valid_x.pt")
torch.save(valid_y, data_path + "valid_y.pt")
torch.save(test_x, data_path + "test_x.pt")
torch.save(test_y, data_path + "test_y.pt")


In [20]:
#実験
b = torch.tensor([1,2,3,4])
a = np.array([1,2,3,4])
print(type(a))
print(type(b))

np.mean(a)

<class 'numpy.ndarray'>
<class 'torch.Tensor'>


2.5

In [25]:
#実験
vec_list = []
vec_list += [1,2,3]
print(type(vec_list))
vec_list = np.array(vec_list)
print(type(vec_list))
features =  torch.from_numpy(vec_list)


<class 'list'>
<class 'numpy.ndarray'>


In [2]:
# coding: utf-8
import sys
sys.path.append('..')
from common import config
config.GPU = True

from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from common.util import eval_perplexity, to_gpu
from dataset import ptb


# ハイパーパラメータの設定
batch_size = 20
wordvec_size = 650
hidden_size = 650
time_size = 35
lr = 20.0
max_epoch = 40
max_grad = 0.25
dropout = 0.5

# 学習データの読み込み
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_val, _, _ = ptb.load_data('val')
corpus_test, _, _ = ptb.load_data('test')

if config.GPU:
    corpus = to_gpu(corpus)
    corpus_val = to_gpu(corpus_val)
    corpus_test = to_gpu(corpus_test)

vocab_size = len(word_to_id)
xs = corpus[:-1]
ts = corpus[1:]

model = BetterRnnlm(vocab_size, wordvec_size, hidden_size, dropout)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

best_ppl = float('inf')
for epoch in range(max_epoch):
    trainer.fit(xs, ts, max_epoch=1, batch_size=batch_size,
                time_size=time_size, max_grad=max_grad)

    model.reset_state()
    ppl = eval_perplexity(model, corpus_val)
    print('valid perplexity: ', ppl)

    if best_ppl > ppl:
        best_ppl = ppl
        model.save_params()
    else:
        lr /= 4.0
        optimizer.lr = lr

    model.reset_state()
    print('-' * 50)


# テストデータでの評価
model.reset_state()
ppl_test = eval_perplexity(model, corpus_test)
print('test perplexity: ', ppl_test)


ImportError: cannot import name 'config' from 'common' (/Users/inoueyuuka/Desktop/kobo2020/common/__init__.py)